# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Building OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

# Download Required Files

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ZcIwWgG1ltOb3G2jfZh_9qHCG2pHbriN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ZcIwWgG1ltOb3G2jfZh_9qHCG2pHbriN" -O images.zip && rm -rf /tmp/cookies.txt
!unzip *file path*

# Extracting Keypoitns

In [ ]:
%cd openpose
!./build/examples/openpose/openpose.bin --image_dir "path of img dir" --write_json "output dir" --display 0 --render_pose 0

# Convert Keypoint to Desired Format

In [ ]:
import glob
import json
import pandas as pd

df_list = []
final_list = []
lst = [{'person_id': [-1], 'pose_keypoints_2d': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'face_keypoints_2d': [], 'hand_left_keypoints_2d': [], 'hand_right_keypoints_2d': [], 'pose_keypoints_3d': [], 'face_keypoints_3d': [], 'hand_left_keypoints_3d': [], 'hand_right_keypoints_3d': []}]


ffiles = glob.glob('/content/output_jsons/**/*.json', 
                   recursive = True)

for sfile in ffiles:
  json_data = json.load(open(sfile))
  mylst = json_data['people']
  df_list.append(mylst)


for item in df_list:
	if len(item) == 0:
		final_list.append(lst)
	else:
		final_list.append(item)
  

for values in final_list:
  print(values[0]['pose_keypoints_2d'])

# Export Keypoints CSV

In [ ]:
import csv

myFile = open('keypoints.csv', 'w')
writer = csv.writer(myFile)
writer.writerow(['pose_keypoints_2d'])

for data_list in final_list:
    writer.writerow(data_list[0]['pose_keypoints_2d'])
    
myFile.close()
myFile = open('keypoints.csv', 'r')
print(myFile.read())
myFile.close()

# Process Labels CSV

In [ ]:
import csv

with open('/content/labels.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

myFile = open('labels.csv', 'w')
writer = csv.writer(myFile)
writer.writerow(['file_name', 'labels'])

for data_list in data:
    writer.writerow(data_list)

myFile.close()
myFile = open('labels.csv', 'r')
print(myFile.read())
myFile.close()